In [4]:
!pip install numpy --user
!pip install pandas --user

In [5]:
import numpy as np
import pandas as pd
import string
import re

In [ ]:
with open('businessCard.txt',mode='r',encoding='utf8',errors='ignore') as f:
    text = f.read()

In [ ]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))

In [ ]:
df = pd.DataFrame(data[1:],columns=data[0])

In [ ]:
df.head(10)

### Cleaning Text
- Remove white space
- Remove Unwanted special characters

In [ ]:
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)


In [ ]:
df['text'] = df['text'].apply(cleanText)

In [ ]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

In [ ]:
dataClean.head(10)

### Convert Data into Spacy Format

In [ ]:
group = dataClean.groupby(by='id')

In [ ]:
cards = group.groups.keys()

In [ ]:
allCardsData = []
for card in cards:
    cardData = []
    grouparray = group.get_group(card)[['text','tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != 'O':
            annot = (start,end-1,label)
            annotations['entities'].append(annot)

        content = content + text + ' '
        
        
    cardData = (content,annotations)
    allCardsData.append(cardData)

In [ ]:
allCardsData

In [ ]:
card_data_df = pd.DataFrame(allCardsData,columns=['text','labels'])
card_data_df['isNull'] = card_data_df['labels'].apply(lambda x: 'Null' if len(x['entities']) ==0 
                                                      else 'Clean')

#### Null entries need to drop

In [ ]:
card_data_df.query('isNull == "Null"')

#### Consilder only clean data

In [ ]:
card_data_df.dropna(inplace=True)
clean_data = card_data_df.query('isNull == "Clean"')[['text','labels']]

In [ ]:
allCardsData = list(map(lambda x: tuple(x), clean_data.values.tolist()))

## Split the Data into Training and Testing Set

In [ ]:
import random

In [ ]:
random.shuffle(allCardsData)

In [ ]:
len(allCardsData)

In [ ]:
TrainData = allCardsData[:240]
TestData = allCardsData[240:]

In [ ]:
import pickle

In [ ]:
pickle.dump(TrainData,open('./data/TrainData.pickle',mode='wb'))
pickle.dump(TestData,open('./data/TestData.pickle',mode='wb'))